Imputing after train split

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
train = train.drop('id', axis = 1)
test = test.drop('id', axis = 1)

In [4]:
pd.set_option('display.max_columns', None)

In [5]:
def replace_non_alpha_with_nan(df):
    cols_to_filter = ['cap-shape', 'cap-surface', 'cap-color', 
                      'does-bruise-or-bleed', 'gill-attachment', 
                      'gill-spacing', 'gill-color', 'stem-surface', 
                      'stem-color', 'has-ring', 'ring-type', 'habitat', 'stem-root', 'veil-type', 'veil-color', 'spore-print-color']

    alphabet_list = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 
                     'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

    col_values = {}
    for col in cols_to_filter:
        value_counts = train[col].value_counts()
        col_values[col] = value_counts[value_counts > 10].index.values.tolist()


    def filter_alpha(value, value_list_no_outliers):
        if isinstance(value, str):
            return value if len(value) == 1 and value in value_list_no_outliers and value in alphabet_list else np.nan # if value is a single character
        
        return np.nan
    
    for col in cols_to_filter:
        df[col] = df[col].apply(lambda x : filter_alpha(x, col_values[col]))

    return df

In [6]:
train = replace_non_alpha_with_nan(train)
test = replace_non_alpha_with_nan(test)

In [7]:
cat_cols = [col for col in train.select_dtypes('object').columns if col != 'class']
num_cols = [col for col in train.select_dtypes('number').columns]
print(f'Categorical columns:\n {cat_cols}\n')
print(f'Numeric columns:\n {num_cols}')

Categorical columns:
 ['cap-shape', 'cap-surface', 'cap-color', 'does-bruise-or-bleed', 'gill-attachment', 'gill-spacing', 'gill-color', 'stem-root', 'stem-surface', 'stem-color', 'veil-type', 'veil-color', 'has-ring', 'ring-type', 'spore-print-color', 'habitat', 'season']

Numeric columns:
 ['cap-diameter', 'stem-height', 'stem-width']


In [8]:
from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer

def encode_train_data(df, num_cols, cat_cols):

    numeric_transformer = Pipeline(steps = [
        ('imputer', KNNImputer(n_neighbors = 3))
    ])

    categorical_transformer = Pipeline(steps = [
        # ('imputer', SimpleImputer(strategy = 'most_frequent')),
        ('ordinal', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)),
        ('adjust', FunctionTransformer(lambda x : x + 1)) # * Adjust function
    ])

    preprocessor = ColumnTransformer(
        transformers = [
            ('num', numeric_transformer, num_cols),
            ('cat', categorical_transformer, cat_cols)
        ]
    )

    df_transformed = pd.DataFrame(preprocessor.fit_transform(df[num_cols + cat_cols]), columns = num_cols + cat_cols)

    df_final = df[['class']].join(df_transformed)

    return df_final

In [9]:
def encode_test_data(df, num_cols, cat_cols):

    numeric_transformer = Pipeline(steps = [
        ('imputer', KNNImputer(n_neighbors=3))
    ])

    categorical_transformer = Pipeline(steps = [
        # ('imputer', SimpleImputer(strategy='most_frequent')),
        ('ordinal', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value = -1)),
        ('adjust', FunctionTransformer(lambda x : x + 1)) # * Adjust function
    ])

    preprocessor = ColumnTransformer(
        transformers = [
            ('num', numeric_transformer, num_cols),
            ('cat', categorical_transformer, cat_cols)
        ]
    )

    df_transformed = pd.DataFrame(preprocessor.fit_transform(df[num_cols + cat_cols]), columns = num_cols + cat_cols)

    df_final = df_transformed
    return df_final

In [10]:
import itertools
from sklearn.preprocessing import LabelEncoder

def find_train_combinations(train, cat_cols, num_cols):
    
    all_columns = cat_cols + num_cols

    # ? returning features from train_combinations with correlations greater than the mean of the original
    
    ord_enc = LabelEncoder()
    train['class'] = ord_enc.fit_transform(train['class'])

    corr_matrix = train.corr()
    
    threshold = abs(corr_matrix['class']).sort_values(ascending=False).mean()
    print(f" Mean Correlation of Original Data {threshold}")


    filtered_cols = [col for col in all_columns if col != 'class']
    print(filtered_cols)
    print(train.columns)
    combinations = itertools.combinations(filtered_cols, 2)
    print(combinations)

    train_combinations = train['class'].to_frame()

    for col1, col2 in combinations:
       combination = train[col1] * train[col2]
       train_combinations = train_combinations.join(combination.rename(f'{col1} x {col2}'))
    
    # ? returning features from train_combinations with correlations greater than the mean of the original

    # corr_combinations = train_combinations.corr()
    # abs_values = abs(corr_combinations['class'])
    # new_cols = abs_values.loc[abs_values > threshold].index.tolist()
    # if 'class' in new_cols:
    #     new_cols.remove('class')
    new_cols = ['gill-attachment x stem-color']
    
    
    train['class'] = ord_enc.inverse_transform(train['class'])
    return train.join(train_combinations[new_cols])


In [11]:
def find_test_combinations(test, cat_cols, num_cols, train_columns):

    all_columns = cat_cols + num_cols

    filtered_cols = [col for col in all_columns if col != 'class']
    print(filtered_cols)
    combinations = itertools.combinations(filtered_cols, 2)

    test_combinations = pd.DataFrame(index = test.index)

    for col1, col2 in combinations:
       combination = test[col1] * test[col2]
       test_combinations = test_combinations.join(combination.rename(f'{col1} x {col2}'))
    
    # ? Remove 'class' feature from test set
    train_columns = train_columns.drop('class')

    test = test.join(test_combinations)

    return test[train_columns]



In [12]:
# train = find_train_combinations(train, cat_cols, num_cols) # ! Change train_new back to train after testing
# test = find_test_combinations(test, cat_cols, num_cols, train.columns)


In [13]:
def handle_missing_data(df_transformed):
    
    df_transformed = df_transformed.fillna(-10)

    print("Missing values after imputation:")
    print(df_transformed.isnull().sum())
    return df_transformed

In [14]:
def drop_columns(df):
    df = df.drop('veil-type', axis = 1)

    return df

In [15]:
# train = drop_columns(train)
# test = drop_columns(test)

In [16]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train['class'] = le.fit_transform(train['class'])

In [17]:
y = train['class']
X = train.drop('class', axis = 1)

In [18]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [27]:
X_train = encode_test_data(X_train, num_cols, cat_cols)
X_test = encode_test_data(X_test, num_cols, cat_cols)


test = encode_test_data(test, num_cols, cat_cols)

In [ ]:
from sklearn.metrics import matthews_corrcoef

def mcc_metric(y_pred, dmatrix):
    y_true = dmatrix.get_label()
    y_pred = (y_pred > 0.5).astype(int) 
    mcc = matthews_corrcoef(y_true, y_pred)
    return 'mcc', mcc

In [ ]:
from sklearn.metrics import matthews_corrcoef
from xgboost import XGBClassifier
import gc

model = XGBClassifier(

    colsample_bytree = 0.6,
    max_depth = 14,
    min_child_weight = 7,
    random_state = 42,
    n_estimators = 200,
)

In [ ]:
XGB = model.fit(
    X_train, 
    y_train, 
    eval_set = [(X_test, y_test)],
    eval_metric = mcc_metric)

c:\Users\hugph\ML\s4e8\venv\Lib\site-packages\xgboost\sklearn.py:889: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-logloss:0.46819	validation_0-mcc:0.91116
[1]	validation_0-logloss:0.33166	validation_0-mcc:0.96217
[2]	validation_0-logloss:0.24768	validation_0-mcc:0.97006
[3]	validation_0-logloss:0.18835	validation_0-mcc:0.97555
[4]	validation_0-logloss:0.14836	validation_0-mcc:0.97691
[5]	validation_0-logloss:0.11745	validation_0-mcc:0.97936
[6]	validation_0-logloss:0.09547	validation_0-mcc:0.98036
[7]	validation_0-logloss:0.08231	validation_0-mcc:0.98061
[8]	validation_0-logloss:0.07016	validation_0-mcc:0.98135
[9]	validation_0-logloss:0.06168	validation_0-mcc:0.98162
[10]	validation_0-logloss:0.05512	validation_0-mcc:0.98204
[11]	validation_0-logloss:0.05166	validation_0-mcc:0.98210
[12]	validation_0-logloss:0.04879	validation_0-mcc:0.98211
[13]	validation_0-logloss:0.04604	validation_0-mcc:0.98235
[14]	validation_0-logloss:0.04339	validation_0-mcc:0.98291
[15]	validation_0-logloss:0.04188	validation_0-mcc:0.98309
[16]	validation_0-logloss:0.04082	validation_0-mcc:0.98319
[17]	va

In [ ]:
y_pred = XGB.predict(X_test)

In [ ]:
score = matthews_corrcoef(y_test, y_pred)
print('MCC', score)

MCC 0.983739054353024


In [ ]:
test_pred_prob = XGB.predict(test)

In [ ]:
test_pred_class = le.inverse_transform(test_pred_prob)

In [ ]:
submission = pd.read_csv('sample_submission.csv')

submission['class'] = test_pred_class

In [ ]:
submission.to_csv('submission.csv', index = False)